In [101]:
%gui qt

from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec


# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

display.EraseAll()
display.View_Iso()
display.FitAll()

pyqt5 backend - Qt version 5.15.2


In [102]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
# shape = read_step_file("../data/0444-1 ANGLED.step")
# shape = read_step_file("../data/ncTest.step")
# shape = read_step_file("../data/TestEA.step")
shape = read_step_file("../data/TestUEA.step")
# shape = read_step_file("../data/TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [103]:
def assign_local_axes_from_extents(axes, half_extents):
    """
    Assign local x, y, z axes based on geometry (not global orientation).

    Parameters:
    - axes: list of 3 gp_Vec or gp_XYZ from OBB
    - half_extents: list of 3 floats representing OBB half-extents along each axis

    Returns:
    - (xaxis, yaxis, zaxis): gp_Vec objects ordered as (web, flange, length)
    - (web_idx, flange_idx, length_idx): the index mapping used
    """

    # Ensure vectors are gp_Vec (convert from gp_XYZ if needed)
    axes_vec = [gp_Vec(ax) if not isinstance(ax, gp_Vec) else ax for ax in axes]

    # Sort half-extents and get index mapping
    sorted_indices = np.argsort(half_extents)
    web_idx, flange_idx, length_idx = sorted_indices

    # Assign local axes based on geometry size
    xaxis = axes_vec[web_idx]     # Web (narrowest)
    yaxis = axes_vec[flange_idx]  # Flange (medium)
    zaxis = axes_vec[length_idx]  # Length (longest)

    # Optionally normalize (you can remove if already normalized)
    xaxis.Normalize()
    yaxis.Normalize()
    zaxis.Normalize()

    return (xaxis, yaxis, zaxis), (web_idx, flange_idx, length_idx)


In [104]:
# Viewer helper function

from OCC.Core.gp import gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# Helper: draw an axis arrow
def draw_axis_arrow(display, origin, direction, scale=100, color='BLACK'):
    end = gp_Pnt(
        origin.X() + direction.X() * scale,
        origin.Y() + direction.Y() * scale,
        origin.Z() + direction.Z() * scale,
    )
    edge = BRepBuilderAPI_MakeEdge(origin, end).Edge()
    display.DisplayShape(edge, color=color, update=False)

# Helper: draw axes L (blue), F (green), W (red)
def draw_axes(display, origin, axes, scale=100):
    L, F, W = axes
    draw_axis_arrow(display, origin, gp_Dir(*L), scale=scale, color='BLUE')
    draw_axis_arrow(display, origin, gp_Dir(*F), scale=scale, color='GREEN')
    draw_axis_arrow(display, origin, gp_Dir(*W), scale=scale, color='RED')

# Helper: reset viewer and show a shape
def reset_view(display, shape, color='YELLOW'):
    display.EraseAll()
    display.DisplayShape(shape, color=color, update=True)
    display.View_Iso()
    display.FitAll()

# Helper: visualize axes and origins
def show_alignment(display, solid, result):
    display.DisplayShape(solid, transparency=0.5)
    
    # Axes and origins
    colors = {'V': 'RED', 'O': 'BLUE', 'U': 'GREEN'}
    for face, origin in [('V', result['origin_v']), ('O', result['origin_o']), ('U', result['origin_u'])]:
        display.DisplayShape(gp_Pnt(*origin), update=True, color=colors[face])
        
        x_dir, y_dir = result['face_axes'][face]
        origin_pnt = gp_Pnt(*origin)
        x_vec = gp_Vec(*x_dir) * 50
        y_vec = gp_Vec(*y_dir) * 50
        display.DisplayVector(x_vec, origin_pnt, color='YELLOW')
        display.DisplayVector(y_vec, origin_pnt, color='CYAN')

# === Helper: Draw OBB wireframe box ===
def draw_obb(display, center, half_extents, axes, color="YELLOW"):
    c = np.array([center.X(), center.Y(), center.Z()])
    axis_vecs = [np.array([a.X(), a.Y(), a.Z()]) for a in axes]

    corners = []
    for dx in (-1, 1):
        for dy in (-1, 1):
            for dz in (-1, 1):
                offset = (
                    dx * half_extents[0] * axis_vecs[0] +
                    dy * half_extents[1] * axis_vecs[1] +
                    dz * half_extents[2] * axis_vecs[2]
                )
                pt = gp_Pnt(*(c + offset))
                corners.append(pt)

    edges = [
        (0, 1), (0, 2), (0, 4),
        (1, 3), (1, 5),
        (2, 3), (2, 6),
        (3, 7),
        (4, 5), (4, 6),
        (5, 7),
        (6, 7)
    ]
    for i, j in edges:
        edge = BRepBuilderAPI_MakeEdge(corners[i], corners[j]).Edge()
        display.DisplayShape(edge, color=color, update=False)


In [105]:
from OCC.Core.gp import gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge
from OCC.Core.Bnd import Bnd_OBB
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
import numpy as np

# Display solid
display.DisplayShape(solid, update=True)

# Compute OBB
obb = Bnd_OBB()
brepbndlib.AddOBB(solid, obb, True, True, False)

center = obb.Center()
half_extents = [obb.XHSize(), obb.YHSize(), obb.ZHSize()]
axes = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]

print("OBB center:", center.X(), center.Y(), center.Z())
print("Half-extents:", half_extents)
for i, axis in enumerate(axes):
    print(f"Axis {i} → ({axis.X():.3f}, {axis.Y():.3f}, {axis.Z():.3f})")

# Draw bounding box
draw_obb(display, center, half_extents, axes)

# Assign local axes based on geometry
(xaxis, yaxis, zaxis), (web_idx, flange_idx, length_idx) = assign_local_axes_from_extents(axes, half_extents)


# Draw local axes
origin = gp_Pnt(center.X(), center.Y(), center.Z())
draw_axis_arrow(display, origin, xaxis, 100, color='RED')    # Local X (web)
draw_axis_arrow(display, origin, yaxis, 100, color='GREEN')  # Local Y (flange)
draw_axis_arrow(display, origin, zaxis, 100, color='BLUE')   # Local Z (length)

# Log results
print("Local X axis (web):", xaxis.X(), xaxis.Y(), xaxis.Z())
print("Local Y axis (flange):", yaxis.X(), yaxis.Y(), yaxis.Z())
print("Local Z axis (length):", zaxis.X(), zaxis.Y(), zaxis.Z())

display.FitAll()

OBB center: 14.5 14.899999999999952 500.0
Half-extents: [50.00000000000006, 25.0, 500.0]
Axis 0 → (0.000, -1.000, -0.000)
Axis 1 → (1.000, 0.000, 0.000)
Axis 2 → (0.000, -0.000, 1.000)
Many colors for color name BLUE, using first.
Local X axis (web): 1.0 0.0 0.0
Local Y axis (flange): 0.0 -1.0 -0.0
Local Z axis (length): 0.0 -0.0 1.0


In [85]:
# import numpy as np
# from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln
# from OCC.Core.GProp import GProp_GProps
# from OCC.Core.BRepGProp import brepgprop
# from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
# from OCC.Core.TopExp import TopExp_Explorer
# from OCC.Core.TopAbs import TopAbs_EDGE
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
# from OCC.Core.TopoDS import topods

# Redefine compute_section_and_length_and_origin_from_obb
def compute_section_and_length_and_origin_from_obb(solid, center_pnt, half_extents, axes):
    # Convert axes to normalized numpy vectors
    axes_np = [np.array([d.X(), d.Y(), d.Z()], float) for d in axes]

    # Compute spans and assign axes by sorting half-extents
    sorted_indices = np.argsort(half_extents)
    web_idx, flange_idx, long_idx = sorted_indices

    span_web = 2.0 * half_extents[web_idx]
    span_flange = 2.0 * half_extents[flange_idx]
    length = 2.0 * half_extents[long_idx]

    # Principal axes
    L = axes_np[long_idx] / np.linalg.norm(axes_np[long_idx])
    F = axes_np[flange_idx] / np.linalg.norm(axes_np[flange_idx])
    W = axes_np[web_idx] / np.linalg.norm(axes_np[web_idx])

    # Section plane at center
    if hasattr(center_pnt, 'X'):
        center = gp_Pnt(center_pnt.X(), center_pnt.Y(), center_pnt.Z())
    else:
        center = gp_Pnt(*center_pnt)

    plane = gp_Pln(center, gp_Dir(*L))

    # Extract section face from plane cut
    sec = BRepAlgoAPI_Section(solid, plane)
    sec.ComputePCurveOn1(True)
    sec.Approximation(True)
    sec.Build()

    exp_e = TopExp_Explorer(sec.Shape(), TopAbs_EDGE)
    wb = BRepBuilderAPI_MakeWire()
    while exp_e.More():
        wb.Add(topods.Edge(exp_e.Current()))
        exp_e.Next()
    face = BRepBuilderAPI_MakeFace(wb.Wire()).Face()

    # Section area
    gp2 = GProp_GProps()
    brepgprop.SurfaceProperties(face, gp2)
    area = gp2.Mass()

    # Origins: V, O, U corners
    centroid = np.array([center.X(), center.Y(), center.Z()], dtype=float)
    origin_v = centroid + half_extents[long_idx] * L + half_extents[flange_idx] * W - half_extents[web_idx] * F
    origin_o = centroid + half_extents[long_idx] * L - half_extents[flange_idx] * W + half_extents[web_idx] * F
    origin_u = centroid + half_extents[long_idx] * L - half_extents[flange_idx] * W - half_extents[web_idx] * F

    # Face axes: X = into part along -L, Y = upward on each face
    face_axes = {
        'V': (-L,  F),
        'O': (-L,  W),
        'U': (-L, -W),
    }

    return {
        'area':         area,
        'span_flange':  span_flange,
        'span_web':     span_web,
        'length':       length,
        'origin_v':     origin_v,
        'origin_o':     origin_o,
        'origin_u':     origin_u,
        'axes':         (L, F, W),
        'face_axes':    face_axes,
        'plane':        plane,
        'centroid':     centroid,
        'half_extents': half_extents
    }


In [31]:
result = compute_section_and_length_and_origin(solid)

NameError: name 'compute_section_and_length_and_origin' is not defined